# Qredo tokenomics simulation - scenario generation

#### June 2023

## 1. Imports

In [1]:
import os
import pandas as pd
import pickle
import itertools

In [2]:
code_dir = os.path.realpath(os.path.join(os.getcwd(), ".."))

import sys
sys.path.append(code_dir)

from mechaqredo.params import default_params_dict
from mechaqredo.sim import run_param_sweep_sim

from mechaqredo.generate_scenarios import generate_params_dict_for_scenario
from mechaqredo.data_models import build_model_data_dict_samples

## 2. Set up parameters

In [3]:
forecast_length = 365*2
data_dict_n_samples = 50

file = os.path.realpath("../data/balances.csv")
outfolder = os.path.realpath("../data/sim_data")
wallet_df = pd.read_csv(file)
params_dict = default_params_dict(forecast_length)
params_dict["wallet_balances_vec"] = wallet_df["balance"].values

In [4]:
scenarios = ['very bad', 'bad', 'base', 'good', 'very good']
scenario_tuple_list = list(itertools.product(scenarios, scenarios, scenarios))
len(scenario_tuple_list)

125

## 3. Build scenarios and save

In [5]:
scenario_dict_list = []
for token_scenario, usage_scenario, staking_scenario in scenario_tuple_list:
    scenario_params_dict = generate_params_dict_for_scenario(
        params_dict, token_scenario, usage_scenario, staking_scenario
    )
    data_dict_list = build_model_data_dict_samples(
            data_dict_n_samples, forecast_length, scenario_params_dict
        )
    scenario_dict = {
        "token_scenario":token_scenario, 
        "usage_scenario": usage_scenario, 
        "staking_scenario": staking_scenario,
        "params_dict": params_dict,
        "data_dict_list": data_dict_list,
    }
    scenario_dict_list.append(scenario_dict)
output_file = os.path.join(outfolder, f"params_scenarios.pickle")
with open(output_file, "wb") as fp:
    pickle.dump(scenario_dict_list, fp)